# Dokumentasjon av datastrukturering skript

Dette dokumentet beskriver implementeringen av en datastruktureringsfunksjon, `transform(input)`, som aggregerer og behandler data fra flere datakilder. Funksjonen er designet for å håndtere komplekse dataoperasjoner, inkludert spørringer, gruppering og aggregering av data for analyseformål.

## Innholdsfortegnelse
- [Funksjonsoversikt](#funksjonsoversikt)
- [Input-egenskaper](#input-egenskaper)
- [Funksjonskomponenter](#funksjonskomponenter)
  - [Initiering og input-validering](#initiering-og-input-validering)
  - [Konstruksjon av datospørringer](#konstruksjon-av-datospørringer)
  - [Hjelpefunksjoner](#hjelpefunksjoner)
    - [getFieldLabel](#getfieldlabel)
    - [getDistinctValues](#getdistinctvalues)
    - [countDistinctEmployeesAndSumWorkingHours](#countdistinctemployeesandsumworkinghours)
  - [Dataaggregering](#dataaggregering)
  - [Datakilder og endelig aggregering](#datakilder-og-endelig-aggregering)
  - [Forberedelse av HTML-tabell](#forberedelse-av-html-tabell)
- [Returstruktur](#returstruktur)

## Funksjonsoversikt

Funksjonen `transform(input)` er ansvarlig for å aggregerer data fra ulike datakilder basert på oppgitte input-parametere. Funksjonen håndterer følgende hovedoppgaver:
1. Validering av nødvendige input-egenskaper.
2. Konstruksjon av datospørringer for filtrering av data innenfor et spesifisert tidsintervall.
3. Spørring og aggregering av data basert på rad- og kolonnefelt.
4. Telling av distinkte ansatte og summere arbeidstimer for spesifikke datakilder.
5. Forberedelse av data for presentasjon i et HTML-tabellformat.

## Input-egenskaper

Funksjonen forventer følgende input-egenskaper:

- **table**: Primær datakilde.
- **table2**: Sekundær datakilde (valgfri).
- **table3**: Tertiær datakilde (valgfri).
- **table4**: Kvartær datakilde (valgfri).
- **table5**: Kvintær datakilde (valgfri).
- **rowField**: Felt som brukes for radgruppering.
- **columnField**: Felt som brukes for kolonnegruppering.
- **encodedQuery1**: Kodet spørring for filtrering av data i primær datakilde.
- **encodedQuery2**: Kodet spørring for filtrering av data i sekundær datakilde (valgfri).
- **encodedQuery3**: Kodet spørring for filtrering av data i tertiær datakilde (valgfri).
- **encodedQuery4**: Kodet spørring for filtrering av data i kvartær datakilde (valgfri).
- **encodedQuery5**: Kodet spørring for filtrering av data i kvintær datakilde (valgfri).
- **returnPercentage**: Boolsk flagg som indikerer om det skal returneres prosentverdier.
- **startOfInterval**: Startdato for intervallet for filtrering av data.
- **endOfInterval**: Sluttdato for intervallet for filtrering av data.

## Funksjonskomponenter

### Initiering og input-validering

Funksjonen starter med å trekke ut og initiere input-egenskapene, med standardverdier der det er aktuelt. Deretter sjekker den for tilstedeværelse av nødvendige egenskaper (`table` og `rowField`). Hvis noen av disse egenskapene mangler, kaster funksjonen en feil.

```javascript
// Sørg for at nødvendige egenskaper er gitt
if (!dataSource1 || !rowField) {
    throw new Error("Manglende nødvendige input-egenskaper");
}


In [ ]:
function getDistinctValues(fieldName, dataSource, encodedQuery, dateQuery) {
    const gr = new GlideAggregate(dataSource);
    gr.addAggregate('COUNT');
    gr.groupBy(fieldName);
    if (encodedQuery) gr.addEncodedQuery(encodedQuery);
    gr.addEncodedQuery(dateQuery);
    gr.query();

    const values = [];
    while (gr.next()) {
        const value = gr.getDisplayValue(fieldName) || gr.getValue(fieldName) || "(empty)";
        values.push(value);
    }
    return values;
}


## Datospørringer

Funksjonen konstruerer datospørringer basert på egenskapene startOfInterval og endOfInterval. Disse spørringene brukes til å filtrere data innenfor det spesifiserte tidsrommet.

In [ ]:
// Konstruksjon av datospørringene
const fravarDateQuery = [
    `fromdateON${startOfInterval}@${startOfInterval}@${endOfInterval}`,
    `ORtodateON${startOfInterval}@${startOfInterval}@${endOfInterval}`,
    `ORtodate>${endOfInterval}`
].join('^');

const timerDateQuery = `assignmentdateON${startOfInterval}@${startOfInterval}@${endOfInterval}`;


# Hjelpefunksjoner

## Hente navn på felt

Denne hjelpefunksjonen henter etiketten til et spesifisert felt fra en gitt datakilde. Hvis feltet ikke er gyldig, returnerer det feltets navn.

In [2]:
function getFieldLabel(dataSource, fieldName) {
    const gr = new GlideRecord(dataSource);
    return gr.isValidField(fieldName) ? gr.getElement(fieldName).getLabel() : fieldName;
}

## Hente verdier til hver kolonne

Denne funksjonen henter distinkte verdier for et spesifisert felt fra en datakilde, og bruker den relevante datospørringen.

In [3]:
function getDistinctValues(fieldName, dataSource, encodedQuery, dateQuery) {
    const gr = new GlideAggregate(dataSource);
    gr.addAggregate('COUNT');
    gr.groupBy(fieldName);
    if (encodedQuery) gr.addEncodedQuery(encodedQuery);
    gr.addEncodedQuery(dateQuery);
    gr.query();

    const values = [];
    while (gr.next()) {
        const value = gr.getDisplayValue(fieldName) || gr.getValue(fieldName) || "(empty)";
        values.push(value);
    }
    return values;
}


## Hente verdier til å regne prosent

Denne funksjonen teller distinkte ansatte og summerer deres arbeidstimer basert på spesifikke kriterier.

In [4]:
function countDistinctEmployeesAndSumWorkingHours(timerDateQuery, rowField, groupByField) {
    const gr = new GlideAggregate('x_gehas_nisprosjek_timer');
    gr.addQuery('ansatt', '!=', '');
    gr.addEncodedQuery(timerDateQuery);
    gr.addAggregate('SUM', 'workinghours');
    gr.groupBy(rowField);
    if (groupByField) gr.groupBy(groupByField);
    gr.groupBy('ansatt');
    gr.query();

    const data = {};
    while (gr.next()) {
        const rowValue = gr.getDisplayValue(rowField) || gr.getValue(rowField) || "(empty)";
        const groupByValue = groupByField ? (gr.getDisplayValue(groupByField) || gr.getValue(groupByField) || "(empty)") : "(none)";
        const employee = gr.getValue('ansatt');
        const workingHours = parseFloat(gr.getAggregate('SUM', 'workinghours'));

        if (!data[rowValue]) data[rowValue] = {};
        if (!data[rowValue][groupByValue]) {
            data[rowValue][groupByValue] = { employees: {}, totalWorkingHours: 0 };
        }

        data[rowValue][groupByValue].employees[employee] = true;
        data[rowValue][groupByValue].totalWorkingHours += workingHours;
    }

    return Object.keys(data).map(rowValue => ({
        rowValue,
        columns: Object.keys(data[rowValue]).reduce((acc, groupByValue) => {
            acc[groupByValue] = {
                distinctCount: Object.keys(data[rowValue][groupByValue].employees).length,
                totalWorkingHours: data[rowValue][groupByValue].totalWorkingHours
            };
            return acc;
        }, {})
    }));
}


# Dataaggregering

Funksjonen aggregerer data etter rader og eventuelt etter kolonner. Den bruker hjelpefunksjonene for å hente nødvendige verdier og utfører de nødvendige beregningene for hver datakilde.

In [5]:
function aggregateData(rowField, groupByField, dataSource, encodedQuery, fravarDateQuery, timerDateQuery, returnPercentage) {
    const gr = new GlideAggregate(dataSource);
    const sumField = 'antalldager';

    if (returnPercentage) {
        gr.addAggregate('SUM', sumField);
    } else {
        gr.addAggregate('COUNT');
    }

    gr.groupBy(rowField);
    if (groupByField) gr.groupBy(groupByField);
    if (encodedQuery) gr.addEncodedQuery(encodedQuery);
    gr.addEncodedQuery(fravarDateQuery);
    gr.query();

    const distinctEmployeeData = countDistinctEmployeesAndSumWorkingHours(timerDateQuery, rowField, groupByField);
    const employeeCounts = {};
    const workingHoursData = {};

    distinctEmployeeData.forEach(row => {
        employeeCounts[row.rowValue] = {};
        workingHoursData[row.rowValue] = {};

        Object.entries(row.columns).forEach(([column, columnData]) => {
            employeeCounts[row.rowValue][column] = columnData.distinctCount;
            workingHoursData[row.rowValue][column] = columnData.totalWorkingHours;
        });
    });

    const data = [];
    while (gr.next()) {
        const rowValue = gr.getDisplayValue(rowField) || gr.getValue(rowField) || "(empty)";
        const groupByValue = groupByField ? (gr.getDisplayValue(groupByField) || gr.getValue(groupByField) || "(empty)") : "(none)";

        const fravarsDager = returnPercentage ? parseFloat(gr.getAggregate('SUM', sumField)) : parseFloat(gr.getAggregate('COUNT'));
        const antallAnsatte = employeeCounts[rowValue]?.[groupByValue] || 0;
        const totaleTimer = workingHoursData[rowValue]?.[groupByValue] || 0;

        const totaleDager = totaleTimer / 7.5;
        const muligeDager = antallAnsatte * totaleDager;
        const arbeidsFravar = antallAnsatte > 0 ? fravarsDager / muligeDager : 0;

        data.push({
            rowValue,
            groupByValue,
            count: arbeidsFravar.toFixed(2)
        });
    }

    return data;
}


Funksjonen aggregerer data fra flere datakilder og kombinerer resultatene i en samlet struktur. Hver datakilde behandles separat, og resultatene kombineres i et endelig datasett.

In [6]:
const dataSources = [
    { name: dataSource1, encodedQuery: encodedQuery1, label: input.label1 },
    { name: dataSource2, encodedQuery: encodedQuery2, label: input.label2 },
    { name: dataSource3, encodedQuery: encodedQuery3, label: input.label3 },
    { name: dataSource4, encodedQuery: encodedQuery4, label: input.label4 },
    { name: dataSource5, encodedQuery: encodedQuery5, label: input.label5 }
];

const combinedData = dataSources.map(({ name, encodedQuery, label }) => {
    if (!name) return null;
    const data = aggregateData(rowField, groupByColumns, name, encodedQuery, fravarDateQuery, timerDateQuery, returnPercentage);
    return { dataSource: label || name, data };
}).filter(Boolean);

const activeDataSourcesCount = combinedData.length;

ReferenceError: dataSource1 is not defined

# Formatering til HTML-tabell

Til slutt forbereder funksjonen overskrifter og rader for å presentere dataene i et HTML-tabellformat.

In [7]:
const groupByHeaders = groupByColumns ? getDistinctValues(groupByColumns, dataSource1, encodedQuery1, fravarDateQuery) : ["(none)"];
const rowHeaders = getDistinctValues(rowField, dataSource1, encodedQuery1, fravarDateQuery);

const rowFieldLabel = getFieldLabel(dataSource1, rowField);

const groupby_column_headers = groupByHeaders.filter(value => value !== rowFieldLabel);
const groupby_row_headers = [rowFieldLabel].concat(rowHeaders);

const filtered_data_header = [];
if (activeDataSourcesCount > 1) {
    groupby_column_headers.forEach(groupByValue => {
        combinedData.forEach(dataSource => {
            filtered_data_header.push(dataSource.dataSource);
        });
    });
}

const rowData = rowHeaders.map(rowValue => {
    const row = [];
    groupByHeaders.forEach(groupByValue => {
        if (groupByValue !== rowFieldLabel) {
            combinedData.forEach(dataSource => {
                const matchingData = dataSource.data.find(data => data.rowValue === rowValue og data.groupByValue === groupByValue);
                const count = matchingData ? matchingData.count : 0;
                row.push(count);
            });
        }
    });
    return row;
});


Expected ',', got 'og' at file:///repl.tsx:23:94

  ....find(data => data.rowValue === rowValue og data.groupByValue === groupByValue);
                                              ~~: Expected ',', got 'og' at file:///repl.tsx:23:94

  ....find(data => data.rowValue === rowValue og data.groupByValue === groupByValue);
                                              ~~

## Returstruktur

Funksjonen returnerer et objekt som inneholder de forberedte dataene, som inkluderer følgende egenskaper:

- groupby_column_headers: Overskrifter for kolonnegrupperingene.
- groupby_row_headers: Overskrifter for radgrupperingene.
- filtered_data_header: Overskrifter for de filtrerte datakildene.
- row_data: Aggregert data klar til presentasjon i et HTML-tabellformat.

In [8]:
return {
    groupby_column_headers: groupby_column_headers.filter(header => header !== '(none)'),
    groupby_row_headers,
    filtered_data_header,
    row_data: rowData
};

SyntaxError: Illegal return statement